Разбираемся с тем, как можно взаимодействовать с GPT3 от сбера 

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [10]:
model_name = "sberbank-ai/rugpt3large_based_on_gpt2" # прописываем имя до модели 

In [12]:

tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
text = "Александр Сергеевич Пушкин родился в "



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Александр Сергеевич Пушкин родился в 
1799 году в Москве.

В


Задаём  входные параметры/ эмбеддинг

Дописывание текста
-------

In [46]:
text = "2+2 = "

In [47]:
input_ids = tokenizer.encode(text, return_tensors="pt")
print(input_ids)

tensor([[  22,   15,   22, 8484,  225]])


In [48]:
out = model.generate(input_ids, max_new_tokens=10, eos_token_id=50256, pad_token_id=50256)

In [49]:
print(list(map(tokenizer.decode, out)))
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

['2+2 = \n4.\n\n4.\n\n5']
2+2 = 
4.

4.

5


Суммаризация
----

Если в конце текста добавлтяь TL;DR, то нейросеть начинает генерировать краткое содержание

In [72]:
text = """ 
Когда я оплачиваю покупку в магазине, я вижу текст без засечек на экране терминала. У цветочных магазинов — вывески в виде прописного шрифта, как будто их кто то написал пером. Почему так ? Почему цветочные магазины не пишут вывески строгим или жирным шрифтом?
TL;DR

"""

text2 = """ 
В детстве ненавидел художественную литературу. Все книги в которых было хоть что-то с малейшим намеком на сюжет с главными героями — шли мимо. Особенно это касается русской литературы в которой сначала ты читаешь сказки про жадную старуху или хитрую лису и не очень хитрого колобка, а к 10 классу ты уже разбираешь депрессивные книги в которых кто-то да точно умрет.TL;DR
"""

In [74]:
input_ids = tokenizer.encode(text, return_tensors="pt")
out = model.generate(input_ids, max_new_tokens=25, eos_token_id=50256, pad_token_id=50256)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

 
Когда я оплачиваю покупку в магазине, я вижу текст без засечек на экране терминала. У цветочных магазинов — вывески в виде прописного шрифта, как будто их кто то написал пером. Почему так? Почему цветочные магазины не пишут вывески строгим или жирным шрифтом?
TL;DR

В общем, я не знаю, что делать.

UPD:

В ответ на комментарий от :



Попробуем на короткрих текстах

In [70]:
text = "Я не люблю есть красные пельмени, потому что белые пельмени вкусней красных. Об этом мне рассказала Настя. TL;DR"

In [71]:
input_ids = tokenizer.encode(text, return_tensors="pt")
out = model.generate(input_ids, max_new_tokens=25, eos_token_id=50256, pad_token_id=50256)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

Я не люблю есть красные пельмени, потому что белые пельмени вкусней красных. Об этом мне рассказала Настя. TL;DR;

- Я не люблю есть пельмени, потому что они не вкусные. Об этом мне рассказала Настя. T


умеет даже так:
--------

In [79]:
text = "По-русски: 'кот', по-английски:" 

input_ids = tokenizer.encode(text, return_tensors="pt")
out = model.generate(input_ids, max_new_tokens=30, do_sample=False)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


По-русски: 'кот', по-английски: 'кот').

В этом случае, как и в случае с "котятами", "котята" - это не только котята


In [82]:
text = "С Настей я познакомился в октябре 2020, мы сильно подружились, а еще через пол года я начал признавться ей в любви - дурак. TL;DR"

input_ids = tokenizer.encode(text, return_tensors="pt")
out = model.generate(input_ids, max_new_tokens=25, eos_token_id=50256, pad_token_id=50256)
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

С Настей я познакомился в октябре 2020, мы сильно подружились, а еще через пол года я начал признавться ей в любви - дурак. TL;DR;

- я не могу без нее жить, она моя жизнь, я не могу без нее жить, она моя жизнь


Токенизация 
-------

In [83]:
# Изначальные текст
text = "Токенизируй меня" 
# Процесс токенизации с помощьюю токенайзера ruGPT-3
tokens = tokenizer.encode(text, add_special_tokens=False) 
# Обратная поэлементая токенизация
decoded_tokens = [tokenizer.decode([token]) for token in tokens] 

print("text:", text)
print("tokens: ", tokens)
print("decoded tokens: ", decoded_tokens)

text: Токенизируй меня
tokens:  [789, 368, 337, 848, 28306, 703]
decoded tokens:  ['Т', 'ок', 'ени', 'зи', 'руй', ' меня']


In [84]:
# Эти три токена по отдельности не декодируются
print(tokenizer.decode([167]))
#>>> �
print(tokenizer.decode([245]))
#>>> �
print(tokenizer.decode([256]))
#>>> �


# Но вместе они образуют иероглиф
print(tokenizer.decode([167, 245, 256]))
#>>> 撝

�
�
�
撝


Методы генерации текста
---------


In [85]:
text = 'Определение: "Нейронная сеть" - это'
input_ids = tokenizer.encode(text, return_tensors="pt")

Greedy Search
---
Самый простой способ — это аргмаксная генерация (greedy search), когда мы каждый раз выбираем токен, у которого максимальная вероятность. Это не самый лучший метод, так как в таком случае генерация застревает в локальных минимумах и часто выдаёт повторяющиеся фрагменты, например the the the the ...

In [86]:
# Пример аргмаксного сэмплирования
out = model.generate(input_ids, 
                     do_sample=False, 
                     max_length=30)

# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Определение: "Нейронная сеть" - это компьютерная программа, которая позволяет создавать и анализировать нейронные сети. Нейронные сети


Beam search
--------

Чуть более сложный и качественный способ сэмплирования — это beam search. В этом случае на каждом шаге мы выбираем не только один самый вероятный токен, а сразу несколько (beam-size), и дальше продолжаем поиск для каждого из выбранных токенов. Таким образом мы разветвляем пути генерации, получая несколько вариантов сгенерированного текста. В итоге можно выбрать тот вариант, у которого лучшая перплексити (уверенность модели в реалистичности текста). Такая генерация обладает хорошей когерентностью (связностью), но обычно у них не хватает "человечности", они кажутся сухими и скучными. Также это не полностью решает проблему повторяющихся фрагментов генерации.

In [87]:
# Пример генерации с помощью beam-search
out = model.generate(input_ids, 
                     do_sample=False, 
                     num_beams=5, 
                     max_length=30)

# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Определение: "Нейронная сеть" - это сеть, состоящая из множества нейронов, соединенных друг с другом. Нейронная


In [90]:
text = 'Я долго думал и в конце захотел создать свой супер интересный блог в котором буду рассказывать новости TL;DR'
input_ids = tokenizer.encode(text, return_tensors="pt")

In [91]:
# Пример генерации с помощью beam-search
out = model.generate(input_ids, 
                     do_sample=False, 
                     num_beams=5, 
                     max_length=30)

# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Я долго думал и в конце захотел создать свой супер интересный блог в котором буду рассказывать новости TL;DR;DR;DR;DR;DR;


Идея - совместить два метода генерации текста 
---

Сэмплирование с температурой
---

Чтобы добавить тексту непредсказуемости и человечности можно использовать вероятностное сэмплирование с температурой. При такой генерации мы берём не самый вероятный токен, а выбираем его "случайно" с учётом распределения вероятностей. Параметр температуры позволяет контролировать степень рандомности. При нулевой температуре этот метод совпадает с аргмаксным сэмплированием, а при очень большой температуре токены будут выбираться совсем наугад. Обычно хорошо работает температура в диапазоне 0.8 - 2.0. Формула модификации распределения вероятностей очень похожа на формулу распределения Больцмана. Чем выше температура системы тем больше "размазывается" распределение вероятностей её возможных состояний, отсюда слово "температура".

In [99]:
text = 'Любовь это'
input_ids = tokenizer.encode(text, return_tensors="pt")

# Пример вероятностного сэмплирования
out = model.generate(input_ids, 
                     do_sample=True, 
                     temperature=1.9,  # регулируем темпиратуру
                     max_length=30)

# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Любовь это как еда и как музыка)) )<s>
Любовь женщины не знает денег - её не обманует ничего - да?
у всех бывает,


Сэмплирование с ограничением маловероятных токенов (Nucleus Sampling)
---

Для запрета сэмплирования совсем уж некорректных токенов вводят top-k или top-p ограничения. В этом случае генерация тоже происходит случайным образом, но мы заранее отсекаем все маловероятные токены. В случае top-k мы просто зануляем все вероятности кроме k самых больших. А в случае top-p мы оставляем такой минимальный сет токенов, чтобы сумма их вероятностей была не больше p.  Ограничение top-p иногда называют nucleus sampling.

In [100]:
# Пример вероятностного сэмплирвоания с ограничением
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=1.3,
                     top_k=20,
                     top_p=0.8,
                     max_length=30,
                    )
# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Любовь это все!

Как-то вечером на кухне в доме, принадлежавшем, судя по всему, богатой семье, мы сидели в большой столовой


In [104]:
# пробуем суммаризацию
text = """ В детстве ненавидел художественную литературу. Все книги в которых было хоть что-то с малейшим намеком на сюжет с главными героями — шли мимо. Особенно это касается русской литературы в которой сначала ты читаешь сказки про жадную старуху или хитрую лису и не очень хитрого колобка, а к 10 классу ты уже разбираешь депрессивные книги в которых кто-то да точно умрет.

Это не значит, что я ненавидел вообще книги, просто я читал другие. Мне нравился научный сленг и естественно я зачитывался детскими энциклопедиями. Такими чтоб картинок побольше было и текст поинтересней и обязательно про космоc. Планеты, полёты в космос, черные дыры это лайк. В то время я хотел стать астрономом.

Что касается книг про динозавров для меня это было ерундой, пусть динозавры сами про себя и читают, они не имеют ничего общего с настоящим миром.

Только позже я нашел в себе интерес читать романы и некоторые из них мне нравились. Или я просто нашел романы и сюжетные книги, которые мне понравились.

Самое главное, что я понял так это то что никто никогда не сможет заставить вас насильно почитать что-то пока это вам самим не станет интересно или нужно. C книгами, как и с фильмами — трудно заставить себя смотреть черно белое кино если у тебя нет никакого интереса к нему.

И если книга вам не нравится никогда не стоит боятся бросить её на середине. Глупо дочитывать не интересную книгу до конца только для галочки.
TL;DR

"""
input_ids = tokenizer.encode(text, return_tensors="pt")

In [105]:
# Пример вероятностного сэмплирвоания с ограничением
out = model.generate(input_ids, 
                     do_sample=True,
                     temperature=1.3,
                     top_k=20,
                     top_p=0.8,
                     max_length=500,
                    )
# Декодирование токенов
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 В детстве ненавидел художественную литературу. Все книги в которых было хоть что-то с малейшим намеком на сюжет с главными героями — шли мимо. Особенно это касается русской литературы в которой сначала ты читаешь сказки про жадную старуху или хитрую лису и не очень хитрого колобка, а к 10 классу ты уже разбираешь депрессивные книги в которых кто-то да точно умрет.

Это не значит, что я ненавидел вообще книги, просто я читал другие. Мне нравился научный сленг и естественно я зачитывался детскими энциклопедиями. Такими чтоб картинок побольше было и текст поинтересней и обязательно про космоc. Планеты, полёты в космос, черные дыры это лайк. В то время я хотел стать астрономом.

Что касается книг про динозавров для меня это было ерундой, пусть динозавры сами про себя и читают, они не имеют ничего общего с настоящим миром.

Только позже я нашел в себе интерес читать романы и некоторые из них мне нравились. Или я просто нашел романы и сюжетные книги, которые мне понравились.

Самое главное,